In [1]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


In [2]:
def classify_text(text, model="llama-3.1-8b-instant"):
    # Define a classification prompt
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are an AI classifier. Classify the following text into one of these categories: ['transportation', 'catering','customer service','security']. Respond with only the category name."
            },
            {
                "role": "user",
                "content": text,  # This is the text that needs classification
            }
        ],
        model=model,
    )

    # Extract the classification result from the model's response
    classification = chat_completion.choices[0].message.content
    return classification


# Example usage: Classify text using the LLaMA model
text_to_classify = "Women - Free Skating"
classification = classify_text(text_to_classify)

print("Classification Result:", classification)


Classification Result: transportation


In [3]:
from pydantic import BaseModel
from typing import Dict, List
import re



# Define a Pydantic model for the response
class ClassificationResult(BaseModel):
    category: str
    probability: float

def classify_text_with_all_probabilities(text: str, model="llama-3.1-8b-instant") -> List[Dict[str, float]]:
    # Send request to the Groq API, modify the system message to request probabilities for all categories
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an AI classifier. Classify the following text into the following categories: "
                    "['transportation', 'catering','customer service','security'] and give the probability for each. Respond in this format: "
                    "<category>: <probability>, <category>: <probability>, ..."
                )
            },
            {
                "role": "user",
                "content": text,  # The text that needs classification
            }
        ],
        model=model,
    )

    # Get the response from the model (e.g., "Sales: 0.85, Support: 0.10, Finance: 0.05, Operations: 0.00")
    response = chat_completion.choices[0].message.content.strip()

    # Create a list to hold the final results
    classification_results = []

    # Split the response by commas to get each category and probability pair
    categories_probabilities = response.split(",")

    for category_probability in categories_probabilities:
        try:
            # Split each pair into category and probability
            category, probability = category_probability.split(":")
            # Clean the probability string by removing non-numeric characters
            probability = re.sub(r'[^\d.]', '', probability)  # Remove anything that's not a digit or decimal point

            # Create a ClassificationResult object for each category
            result = ClassificationResult(
                category=category.strip(),
                probability=float(probability.strip())
            )
            # Add the result to the list
            classification_results.append(result.dict())
        except ValueError:
            # Handle case where the response format is not as expected
            continue

    # Return the list of results
    return classification_results

# Example usage
text_to_classify = "Join us for our annual company retreat on November 15th at the Lakeside Resort. Activities include team-building exercises, workshops, and a gala dinner."
classification = classify_text_with_all_probabilities(text_to_classify)

print("Classification Results:", classification)


Classification Results: [{'category': 'transportation', 'probability': 0.0}, {'category': 'catering', 'probability': 0.4}, {'category': 'customer service', 'probability': 0.0}, {'category': 'security', 'probability': 0.0}]
